In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline

In [2]:
df= pd.read_csv('dummy_marketing_mix_data_upd11.csv')
pd.options.display.max_columns = None

In [3]:
df.head()

,Date,inventory,price,loyalty_promotions,Revenue,weather,holiday_flags,discounts,event_1,event_2,promotions,gdp,cci,cpi,competition_A_data,competition_B_data,competition_C_data,competition_D_data,spend_digital_video,spend_digital_display,spend_digital_audio,spend_paid_social,spend_paid_search,spend_tv,spend_radio,spend_print,digital_video_impressions,digital_display_impressions,digital_audio_impressions,paid_social_impressions,paid_search_impressions,paid_search_clicks,affiliates_impressions,affiliates_clicks,email_sent,tv_grps,radio_grps,print_circulation,dm_sent
0,02/01/2022,1100,46.67,1130,215483.58,22.29,0,0.00,0,0,1,1003.070000,-7.84,4.30,27982,33793,26984,34140,9402.35,15206.57,9960.51,16465.77,11122.01,6844.55,16896.31,17697.27,1912040,1175812,715457,1656231,1144590,50537,1286045,60153,11764,108,167,345227,6385
1,09/01/2022,1045,20.92,1235,97948.36,28.33,0,0.00,0,0,0,1014.605367,1.09,3.60,27356,34069,28857,37982,5197.36,8405.77,5505.90,9101.83,6147.94,3783.48,9339.82,9782.57,1368800,1355499,549877,2224207,1421880,44403,1354405,57200,13057,161,177,408916,6619
2,16/01/2022,1014,47.25,1466,69507.17,19.21,0,19.03,0,0,0,1001.053157,8.77,0.38,30079,33499,29216,29344,6894.03,11149.83,7303.29,12073.10,8154.93,5018.59,12388.79,12976.07,1435069,955045,632460,1632524,1008928,61427,1319432,38220,19935,101,217,508197,5221
3,23/01/2022,1057,29.01,1814,106809.39,20.48,0,13.73,0,0,0,1017.466049,6.18,1.17,27507,32171,28685,34903,14026.04,22684.52,14858.67,24562.95,16591.35,10210.42,25205.22,26400.05,1437523,1149448,534271,2140367,1517924,77674,1388966,68977,15239,191,173,353613,8564
4,30/01/2022,901,39.04,1870,94599.93,28.17,0,5.48,0,0,0,1000.377232,-7.88,2.25,23207,28474,29787,35351,12560.36,20314.06,13305.98,21996.19,14857.60,9143.46,22571.35,23641.32,1265255,1173046,680065,2226155,1044726,45013,1108453,69175,15690,199,210,451101,5097


# Function checking for missing values and right format

In [70]:
def checking_func(df, date_format='%m/%d/%y', mandatory_col=None):
    
    print("Checking for missing values... ")
    missing_val= df.isnull().sum()
    if missing_val.any():
        print('Missing values present in: ',missing_val[missing_val>0])
    else:
        print("No missing values found")
        
        
    print("\nChecking for date format... ")
    for col in df.select_dtypes(['object']):
        try:
            pd.to_datetime(df[col], format='%m/%d/%Y',errors= 'raise')
            print(col,"is in the correct format")
        except ValueError:
            print(col,"is not in the right format. Please provide the dates in 'mm/dd/yyyy' format")
            

            for i in range(len(df[col])):
                try:
                    datetime.strptime(df[col].iloc[i],'%m/%d/%Y')
                except(ValueError, TypeError):
                    print("Invalid date format in row ",i,'of the column ',col,': ',df[col].iloc[i])
                    #here should I write the code asking the user to enter the right format date??
                    
                    
    print("\nChecking for mandatory columns and if they are case-sensitive...")
    if mandatory_col is None:
        mandatory_col=['Date','Revenue']
    
    df_cols_lower=[col.lower() for col in df.columns]
    for cols in mandatory_col:
        cols2=cols.lower()
        if cols2 not in df_cols_lower:
            print("Please ensure that ",cols," is present in the dataset!" )
        else:
            if cols not in df.columns:
                print("Make sure the column '",cols2,"' is written as '",cols,"'")
    

In [64]:
checking_func(df)

Checking for missing values... 
No missing values found

Checking for date format... 
date is not in the right format. Please provide the dates in 'mm/dd/yyyy' format
Invalid date format in row  2 of the column  date :  16/01/2022
Invalid date format in row  3 of the column  date :  23/01/2022
Invalid date format in row  4 of the column  date :  30/01/2022
Invalid date format in row  6 of the column  date :  13/02/2022
Invalid date format in row  7 of the column  date :  20/02/2022
Invalid date format in row  8 of the column  date :  27/02/2022
Invalid date format in row  10 of the column  date :  13/03/2022
Invalid date format in row  11 of the column  date :  20/03/2022
Invalid date format in row  12 of the column  date :  27/03/2022
Invalid date format in row  15 of the column  date :  17/04/2022
Invalid date format in row  16 of the column  date :  24/04/2022
Invalid date format in row  19 of the column  date :  15/05/2022
Invalid date format in row  20 of the column  date :  22/05

# EDA part

In [4]:
#Assuming the date column is named as "Date" and revenue column named as "Revenue"

df['Date']=pd.to_datetime(df['Date'])
df.set_index('Date')

ValueError: time data "16/01/2022" doesn't match format "%m/%d/%Y", at position 2. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.